In [12]:
import geopandas as gpd
import rasterio as rio

import rasterstats

import matplotlib.pyplot as plt
from rasterio.plot import show

from os.path import join
from os import listdir
from glob import glob

In [2]:
ROOT_DIR = 'f:/datasets/NATUR_CUNI'
stats = ['mean', 'median', 'std', 'count']

## Testing raster statistics

In [3]:
def compute_zonal_stats_one_orbit(root_dir, raster_dir, stats_gdf, stats=['mean']):
    # Get raster filenames
    raster_dir = join(raster_dir)
    rasters_list = glob('coh_*.tif', root_dir=raster_dir)
    
    print(f'Processing {len(rasters_list)} rasters:')
    # loop through all coherence rasters
    for raster in rasters_list:
        print(raster)
        
        # Read raster array and transform
        raster_path = join(raster_dir, raster)
        with rio.open(raster_path) as src:
            arr = src.read()
            affine = src.transform

        # compute zonal statistics for individual polarizations
        for idx_band, pol in enumerate(('VH', 'VV')):
            prefix = f'{pol}_RO{}_{raster[4:8]}_{raster[8:12]}-{raster[17:21]}_'

            stats_list = rasterstats.zonal_stats(stats_gdf, arr[idx_band,:,:], affine=affine, geojson_out=True, prefix=prefix, stats=stats, nodata=-1)
            stats_gdf = gpd.GeoDataFrame.from_features(stats_list, crs=stats_gdf.crs)

    print('Zonal stats successfuly computed.')
    return stats_gdf

In [4]:
def compute_zonal_stats_all_orbits(year_dir, gdf_vector, stats):
    for ro_dir in os.listdir(year_dir):#glob(f'{year_dir}/*'):
        print(f'Relative orbit: {ro_dir}')
        gdf_vector = compute_zonal_stats_one_orbit(ro_dir, gdf_vector, stats)
    return gdf_vector

In [10]:
def zonal_stats_all(root, stats=['mean'], years=(2021, 2022)):
    for year in years:
        in_vector  = join(ROOT_DIR, f'reference/{year}/vyjezdy_{year}_4326_singlepart.gpkg')
        out_vector = join(ROOT_DIR, f'reference/{year}/vyjezdy_{year}_zonal_stats.gpkg')
        raster_dir = join(ROOT_DIR, f's1_coherence/{year}')
    
        print(f'Year: {year}')
        gdf_in = gpd.read_file(in_vector)
        gdf_out = compute_zonal_stats_all_orbits(raster_dir, gdf_in, stats)
        gdf_out.to_file(out_vector)

In [11]:
zonal_stats_all(ROOT_DIR, stats=stats, years=(2021,))

Year: 2021
Relative orbit: f:/datasets/NATUR_CUNI\s1_coherence/2021\146
Processing 15 rasters:
coh_20210505_20210511.tif
coh_20210511_20210517.tif
coh_20210517_20210523.tif
coh_20210523_20210529.tif
coh_20210529_20210604.tif
coh_20210604_20210610.tif
coh_20210610_20210616.tif
coh_20210616_20210622.tif
coh_20210622_20210628.tif
coh_20210628_20210704.tif
coh_20210704_20210710.tif
coh_20210710_20210716.tif
coh_20210716_20210722.tif
coh_20210722_20210728.tif
coh_20210728_20210803.tif
Zonal stats successfuly computed.
